In [ ]:
import os
import numpy as np
import glob
from fitcurve import *
import matplotlib.pyplot as plt
import sys
def runstats(array,dim,norm):
    count = sum(array)
    [maxextent,avgextent] = findextent(array,dim,count)
    [dif,front,junk1,junk2] = fitcurve(array,dim,norm)
    [area,density] = hullareafind(array,dim,count)
    return [count,area,maxextent,avgextent,dif,front,density]

In [ ]:

#trying to normalise for density by dividing by t0 avgdist

basepath = "[addpath]"
simulationtitle = "[addtitle]"

INDEX = -1 #This index gives the sim ID
INDEX2 = -1 #This index gives the file ID
radius = 5.0
agr=1

#open file and write the header
outfile = open(os.path.join(basepath,simulationtitle+'.csv'),"w+")
writethis = "Simulation,filename,sid,ID,Time,Radius,Agression,Ratio,ProA,ProB,DeaA,DeaB,MigA,MigB,IntA,IntB,SizeA,SizeB,AreaA,AreaB,MaxExtentA,MaxExtentB,AverageExtentA,AverageExtentB,DiffusionA,DiffusionB,FrontA,FrontB,DensityA,DensityB\n"
outfile.write(writethis)
outfile.flush()


#read file list
files = sorted(glob.glob(os.path.join(basepath,"*[addtitle]*")))
for filename in files:
    INDEX2 = INDEX2+1 #increment file ID
    i = 0 #Set time ticker to zero
    size = os.path.getsize(filename) #get file size
    f = open(filename,"rb") #open file
    print(filename,size) #print file names and size to track progress and see issues
    
    while i <= size-1:
        f.seek(i,os.SEEK_SET)
        sid = np.fromfile(f,dtype=np.int8,count=1)[0] #sim id from binary file
        i = i+1
        
        f.seek(i,os.SEEK_SET)
        rates = np.fromfile(f,dtype=np.float64,count=8) #rates from binary file
        i = i+(8*8)
        proA = rates[0]
        proB = rates[1]
        migA = rates[2]
        migB = rates[3]
        deaA = rates[4]
        deaB = rates[5]
        intA = rates[6]
        intB = rates[7]
        
        
        f.seek(i,os.SEEK_SET)
        newratio = np.fromfile(f,dtype=np.float64,count=1)[0] #get the ratio from binary file
        i = i+8
        
        f.seek(i,os.SEEK_SET)
        time = np.fromfile(f,dtype=np.float64,count=1) #get time from binary file
        i = i+8
        time = round(time[0])
        
                
        f.seek(i,os.SEEK_SET)
        array1 = np.fromfile(f,dtype=np.int8,count=(64*64)) #get type 1 array
        i = i+(64*64)
        
        f.seek(i,os.SEEK_SET)
        array2 = np.fromfile(f,dtype=np.int8,count=(64*64)) #get type 2 array
        i = i+(64*64)
        
        if time < 1:
            ratio = newratio
            INDEX = INDEX + 1
            countA = sum(array1)
            countB = sum(array2)
            if ratio == 0.0:
                normA = np.nan
                normB = findnorm(array2,64,countB)
            elif ratio == 1.0:
                normA = findnorm(array1,64,countA)
                normB = np.nan
            else:
                normA = findnorm(array1,64,countA)
                normB = findnorm(array2,64,countB)


        
        if newratio > 0.0: #avoid extra computation from scanning empty array
            [countA,areaA,maxextentA,avgextentA,difA,frontA,densityA] = runstats(array1,64,normA)
        else:
            [countA,areaA,maxextentA,avgextentA,difA,frontA,densityA] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
        if newratio < 1.0: #avoid extra computation from scanning empty array
            [countB,areaB,maxextentB,avgextentB,difB,frontB,densityB] = runstats(array2,64,normB)
        else:
            [countB,areaB,maxextentB,avgextentB,difB,frontB,densityB] = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
        
        
        
        writethis = simulationtitle+","+str(filename)+","+str(sid)+","+str(INDEX)+","+str(time)+","+str(radius)+","+str(agr)+","+str(ratio)+","
        writethis = writethis+str(proA)+","+str(proB)+","
        writethis = writethis+str(deaA)+","+str(deaB)+","
        writethis = writethis+str(migA)+","+str(migB)+","
        writethis = writethis+str(intA)+","+str(intB)+","
        writethis = writethis+str(countA)+","+str(countB)+","
        writethis = writethis+str(areaA)+","+str(areaB)+","
        writethis = writethis+str(maxextentA)+","+str(maxextentB)+","
        writethis = writethis+str(avgextentA)+","+str(avgextentB)+","
        writethis = writethis+str(difA)+","+str(difB)+","
        writethis = writethis+str(frontA)+","+str(frontB)+","
        writethis = writethis+str(densityA)+","+str(densityB)+"\n"
        outfile.write(writethis)
        outfile.flush()
        
outfile.close()